In [5]:
import os
from groq import Groq
import json
import time

In [ ]:
client = Groq(api_key="GROQ_API_KEY")

In [7]:
def analyze_query(user_query):
    """
    Analyzes the query and returns a JSON with intent and extracted data in a single pass.
    """
    
    system_prompt = """
    You are an intelligent extraction engine for a real estate chatbot.
    Analyze the user's input and output a STRICT JSON object based on the intent.

    ### 1. INTENT: DISCOVERY
    - Trigger: User looks for apartments (search, find, available, 'I want a flat...').
    - Extraction Rules:
      * "location": MANDATORY. The specific street, neighborhood, or city mentioned. If none, set to null.
      * "room_type": Optional. (e.g., "1 bedroom", "studio", "penthouse"). Null if not specified.
      * "amenities": Optional List. (e.g., ["gym", "pool", "wifi"]). Empty list [] if not specified.
    
    ### 2. INTENT: DETAILS
    - Trigger: User asks specific questions about a NAMED apartment.
    - Extraction Rules:
      * "apartment_name": The specific name of the building/unit (e.g., "Splendom Atocha", "Splendom Chacao").
    
    ### 3. INTENT: OTHER
    - Trigger: Greetings, math questions, out-of-scope topics, or general chatter.
    - Extraction Rules:
      * "default_reply": A polite standard message IN ENGLISH stating you can only assist with apartment information.

    ### JSON OUTPUT SCHEMA
    You must return a single JSON object with this exact structure. Set fields to null if they don't apply to the active intent.
    
    {
      "intent": "DISCOVERY" | "DETAILS" | "OTHER",
      "discovery_data": {
        "location": "string or null",
        "room_type": "string or null",
        "amenities": ["string"] 
      },
      "details_data": {
        "apartment_name": "string or null"
      },
      "other_data": {
        "reply": "string or null"
      }
    }
    """

    try:
        start_time = time.time()
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ],
            temperature=0, #So it doesn't hallucinate
            response_format={"type": "json_object"}
        )
        end_time = time.time()
        
        print(f"Latency: {end_time - start_time:.4f} seconds")
        
        #Parse JSON string to Python dictionary
        return json.loads(completion.choices[0].message.content)

    except Exception as e:
        return {"error": str(e)}

In [ ]:
test_queries = [
    "I am looking for a flat in Calle Lagasca with a gym and 1 bedroom",
    "How many square feet does Splendom Chacao have?",
    "How much is 15 multiplied by 27?",
    "I want an apartment near Retiro Park with a swimming pool"
]

for query in test_queries:
    result_json = analyze_query(query)
    print(f"{query} --> {json.dumps(result_json, indent=None)}") 

Latency: 0.4416 seconds
I am looking for a flat in Calle Lagasca with a gym and 1 bedroom --> {"intent": "DISCOVERY", "discovery_data": {"location": "Calle Lagasca", "room_type": "1 bedroom", "amenities": ["gym"]}, "details_data": {"apartment_name": null}, "other_data": {"reply": null}}
Latency: 0.2488 seconds
How many square feet does Splendom Chacao have? --> {"intent": "DETAILS", "discovery_data": {"location": null, "room_type": null, "amenities": []}, "details_data": {"apartment_name": "Splendom Chacao"}, "other_data": {"reply": null}}
Latency: 0.3060 seconds
Explícame la teoría de la relatividad --> {"intent": "OTHER", "discovery_data": {"location": null, "room_type": null, "amenities": []}, "details_data": {"apartment_name": null}, "other_data": {"reply": "I can only assist with apartment information. Please let me know how I can help you with that."}}
Latency: 0.3710 seconds
I want an apartment near Retiro Park with a swimming pool --> {"intent": "DISCOVERY", "discovery_data": {